Agent based simulation from class - see code named hap774-AMB-Peterson_HAll

1. three types of agents
2. Floorplan coded as graphs
3. List of classes offered in building
4. Agents need to be aware of:
    - weekly schedule
    - current location
    - current activity
    - goal related to current activity

In [13]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import pandas as pd

In [16]:
ss = pd.DataFrame([['M 1500', 'M 1615', 'PETRSN 1113', 'ACCT 351'],
['W 1500', 'W 1615', 'PETRSN 1113', 'ACCT 351']],
columns = ['start', 'end', 'location', 'class'])

ss

In [55]:
def next_time(t):
    
    dd = t[0]
    hh = t[2:4]
    mm = t[4:]
    
    newmm = (int(mm) + 1) % 60
    if newmm == 0:
        newhh = (int(hh) + 1) % 24
    else:
        newhh = int(hh)
    
    if newmm < 10:
        newmm = '0' + str(newmm)
    else:
        newmm = str(newmm)
        
    if newhh < 10:
        newhh = '0' + str(newhh)
    else:
        newhh = str(newhh)    
        
    newtt = newhh + newmm    
    
    if newtt == '0000':
        if dd == 'M':
            dd = 'T'
        elif dd == 'T':
            dd = 'W'
        elif dd == 'W':
            dd = 'R'
        elif dd == 'R':
            dd = 'F'
        elif dd == 'F':
            dd = 'S'
        elif dd == 'S':
            dd = 'U'    
        elif dd == 'U':
            dd = 'M'    
    
    ret = dd + ' ' + newtt 
    
    return ret

In [57]:
next_time('M 1500')

'M 1501'

In [78]:
class StudentAgent(Agent):
    """A student agent with """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        #property - a current location or weekly schedule
        #need agent to know what to do based off of current action. what is it doing right now and what to do next
        self.current_location = None
        self.current_activity = None
        self.schedule = ss
        
    def step(self): 
        
        if (self.schedule['start'] == self.model.realtime).astype('int').max() > 0:
            print('entering class' , self.model.realtime)
            self.current_activity = 'in class'
            
        if (self.schedule['end'] == self.model.realtime).astype('int').max() > 0:
            print('exiting class' , self.model.realtime)
            self.current_activity = None
            
        '''
        if self.model.realtime == ss.iloc[0]['start']:
            print('entering class')
        print('my schedule is ', self.schedule)
        pass
        '''

In [84]:
class PetersonModel(Model):
   
    def __init__(self, N):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        
        # keeps track of the simulation real time
        self.realtime = 'M 0000'
        
        # Create agents
        for i in range(self.num_agents):
            a = StudentAgent(i, self)
            self.schedule.add(a)
            
    def step(self):
        '''Advance the model by one step'''
        self.realtime = next_time(self.realtime) #this starts the clock at 0000
        self.schedule.step()
        #self.realtime = next_time(self.realtime) #this starts the clock at 0001
        
        

In [85]:
m= PetersonModel(1)

In [86]:
m.step()

In [87]:
for i in range(10000):
    m.step()

entering class M 1500
exiting class M 1615
entering class W 1500
exiting class W 1615
